In [1]:
!pip install pyenchant

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 3.9 MB/s eta 0:00:00


In [2]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.7 MB/s eta 0:00:00


In [10]:
# Importing Libraries
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import pandas as pd
import nltk
from nltk.corpus import cmudict
import string
import enchant
import textstat
import re
nltk.download('punkt')
nltk.download('cmudict')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [11]:
# Importing and Processing Input Excel File
URL_List = pd.read_excel("Input.xlsx")
urls = URL_List["URL"].tolist()

In [12]:
# Importing Text files for Positive, Negative and Stop Words
with open("positive-words.txt","r",encoding="utf-8", errors="ignore") as file:
    pw = file.read()
    positive_words = pw.split()

with open("negative-words.txt","r",encoding="utf-8", errors="ignore") as file:
    nw = file.read()
    negative_words = nw.split()
    
with open("StopWords_Auditor.txt","r",encoding="utf-8", errors="ignore") as file:
    sa = file.read()
    stopwords_auditor = sa.split()
    
with open("StopWords_Currencies.txt","r",encoding="utf-8", errors="ignore") as file:
    sc = file.read()
    stopwords_currencies = sc.split()

with open("StopWords_DatesandNumbers.txt","r",encoding="utf-8", errors="ignore") as file:
    sdn = file.read()
    stopwords_datesandnumbers = sdn.split()
    
with open("StopWords_Generic.txt","r",encoding="utf-8", errors="ignore") as file:
    sg = file.read()
    stopwords_generic = sg.split()
    
with open("StopWords_GenericLong.txt","r",encoding="utf-8", errors="ignore") as file:
    sgl = file.read()
    stopwords_genericlong = sgl.split()
    
with open("StopWords_Geographic.txt","r",encoding="utf-8", errors="ignore") as file:
    sge = file.read()
    stopwords_geographic = sge.split()
    
with open("StopWords_Names.txt","r",encoding="utf-8", errors="ignore") as file:
    sn = file.read()
    stopwords_name = sn.split()

## Combining all the stopwords
stopwords = stopwords_auditor + stopwords_currencies + stopwords_datesandnumbers + stopwords_generic + stopwords_genericlong + stopwords_geographic + stopwords_name

# Creating an empty list to store the output
output = []

In [14]:
# Data Extraction loop
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    article_text = soup.get_text()
    
    # Calculating Variables
    ## Positive Score
    ps = 0
    for word in article_text.split():
        if word in positive_words:
            ps += 1
    
    ## Negative Score
    ns = 0
    for word in article_text.split():
        if word in negative_words:
            ns += 1
            
    ## Removing Stopwords
    def clean_text(text):
        tokens = nltk.word_tokenize(text)
        clean_tokens = [word.lower() for word in tokens if word.lower() not in stopwords and word.lower() not in string.punctuation]
        return " ".join(clean_tokens)
    cleaned_text = clean_text(article_text)
    
    ## Average Sentence Length
    sentences = nltk.sent_tokenize(cleaned_text)
    words = nltk.word_tokenize(cleaned_text)
    total_words = len(words)
    total_sentences = len(sentences)
    asl = total_words/total_sentences if total_words > 0 else 0

    def is_complex_word(word):
        return nltk.pos_tag([word])[0][1] in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    
    def complex_words(text):
        words = nltk.word_tokenize(text)
        return [word for word in words if is_complex_word(word)]
    
    def get_percentage_complex_words(text):
        d_cmu = cmudict.dict()
        d_enchant = enchant.Dict("en_US")
        words = text.split()
        def syllable(word):
            try:
                return [len(list(y for y in x if y[-1].isdigit())) for x in d_cmu[word]][0]
            except KeyError:
                return 0
        complex_words = [word for word in words if syllable(word) > 2 and d_enchant.check(word)]
        total_words = len(words)
        return 0 if total_words == 0 else len(complex_words) / total_words * 100
    
    pcw = get_percentage_complex_words(cleaned_text)
    
    ## FOG Index
    def flesch_kincaid_index(text):
        words = nltk.word_tokenize(text)
        sentences = nltk.sent_tokenize(text)
        num_words = len(words)
        num_sentences = len(sentences)
        syllables = 0
        for word in words:
            syllables += textstat.syllable_count(word)
        avg_sent_len = num_words / num_sentences
        avg_syllables_per_word = syllables / num_words
        return 0.39 * avg_sent_len + 11.8 * avg_syllables_per_word - 15.59
    fogi = flesch_kincaid_index(cleaned_text)
    
    ## Average of Words per sentence
    words = nltk.word_tokenize(cleaned_text)
    total_words = len(words)
    sentences = nltk.sent_tokenize(cleaned_text)
    total_sentences = len(sentences)
    asl = total_words/total_sentences if total_words > 0 else 0
    
    ## Complex Word Count  
    def get_complex_word_count(text):
        d = cmudict.dict()
        d_enchant = enchant.Dict("en_US")
        def syllable(word):
            try:
                return [len(list(y for y in x if y[-1].isdigit())) for x in d[word]][0]
            except KeyError:
                return 0
        words = text.split()
        complex_words = [word for word in words if syllable(word) > 2 and d_enchant.check(word)]
        return len(complex_words)
    comp = get_complex_word_count(cleaned_text)
    
    # Syllable per word calculation
    def count_syllables(word):
        vowels = "aeiouyAEIOUY"
        count = 0
        word = word.lower()
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith("e"):
            count -= 1
        if word.endswith("le"):
            count+=1
        if count == 0:
            count +=1
        return count
    syllable = count_syllables(cleaned_text)

    # Average word length calculation
    def average_word_length(text):
        words = nltk.word_tokenize(text)
        total_letters = sum(len(word) for word in words)
        return total_letters / len(words) if len(words) > 0 else 0

    # Pronoun counting 
    def count_personal_pronouns(text):
      personal_pronouns = "I|me|you|he|him|she|her|it|we|us|they|them"
      count = len(re.findall(f'\\b({personal_pronouns})\\b', text))
      return count

    pronoun_count = count_personal_pronouns(cleaned_text)

    # Calculate average word length
    awl = average_word_length(cleaned_text)
        
    # Performing Text Analysis
    blob = TextBlob(article_text)
    
    # Compute the Variables
    positive_score = ps
    negative_score = ns
    polarity_score = blob.sentiment.polarity
    subjectivity_score = blob.sentiment.subjectivity
    avg_sentence_length = asl
    percentage_of_complex_words = pcw
    fog_index = fogi
    avg_number_of_words_per_sentence = asl
    complex_word_count = comp
    word_count = len(blob.words)
    syllable_per_word = syllable
    personal_pronouns = pronoun_count
    avg_word_length = awl

    # Saving the output for each URL
    output.append([url, positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length,
              percentage_of_complex_words, fog_index, avg_number_of_words_per_sentence, complex_word_count, word_count,
              syllable_per_word, personal_pronouns, avg_word_length])

In [15]:
# Exporting the Output as Excel file
df = pd.DataFrame(output, columns=["URL", "positive_score", "negative_score", "polarity_score", "subjectivity_score", 
                                   "avg_sentence_length", "percentage_of_complex_words", "fog_index", 
                                   "avg_number_of_words_per_sentence", "complex_word_count", "word_count", 
                                   "syllable_per_word", "personal_pronouns", "avg_word_length"])
df.to_excel("output.xlsx", index=False)